## Block Matching

In [1]:
# Importing libraries
import numpy as np
from PIL import Image
import cv2

# Defining Variables
left_images = ['material/l1.png', 'material/l2.png', 'material/l3.png']
right_images = ['material/r1.png', 'material/r2.png', 'material/r3.png']
windows = [21]
methods = ['SAD', 'SSD']

# Defining Functions
def calculate_disparity(left_image, right_image, window_size, method):

    # get the shape of the images
    left_shape = left_image.shape

    # get the height and width of the images
    height = left_shape[0]
    width = left_shape[1]
    disparity_map = np.zeros((height, width))

    # calculate the disparity map
    for i in range(window_size//2, height-window_size//2):
        for j in range(window_size//2, width-window_size//2):
            left_block = left_image[i-window_size//2:i+window_size//2+1, j-window_size//2:j+window_size//2+1]
            cost, best_cost, best_match = 0, np.inf, 0
            for k in range(window_size//2, width-window_size//2):
                right_block = right_image[i-window_size//2:i+window_size//2+1, k-window_size//2:k+window_size//2+1]
                if method == 'SAD':
                    cost = np.sum(np.abs(left_block - right_block))
                elif method == 'SSD':
                    cost = np.sum(np.square(left_block - right_block))
                if cost < best_cost:
                    best_cost = cost
                    best_match = k

            # calculate the disparity
            disparity_map[i, j] = np.abs(j - best_match)

    # Normalizing the disparity map using min-max normalization
    normalized_disparity_map = (disparity_map - np.min(disparity_map)) / (np.max(disparity_map) - np.min(disparity_map)) * 255
    return disparity_map, normalized_disparity_map

# Calculating the disparity map for each image with each window size and method
for image_index in range(0,len(left_images)):
    left_img = np.asarray(Image.open(left_images[image_index]).convert('RGB'))
    right_img = np.asarray(Image.open(right_images[image_index]).convert('RGB'))
    for window in windows:
        for method in methods:
            disparity_map, normalized_disparity_map  = calculate_disparity(left_img, right_img, window, method)
            cv2.imwrite('results/{}/disparity_map_window_size_{}_{}.png'.format(image_index+1, window, method), disparity_map.astype("uint8"), [cv2.IMWRITE_PNG_COMPRESSION, 0])
            cv2.imwrite('results/{}/normalized_disparity_map_window_size_{}_{}.png'.format(image_index+1, window, method), normalized_disparity_map.astype("uint8"), [cv2.IMWRITE_PNG_COMPRESSION, 0])

## Dynamic Programming